In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from lightning import LightningModule, Trainer
import lightning as L
from torch.nn.parameter import Parameter
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import seaborn as sb
import os
from torch.nn import Linear, Tanh
import pickle as pkl

# Data Processing

In [2]:
file = r'.\Dataset\datav1.pkl'
with open(file , 'rb') as f:
    rdata = pkl.load(f)

type(rdata), rdata.shape, rdata.dtype

(torch.Tensor, torch.Size([50000, 256]), torch.float32)

In [3]:
precision = 32
if precision == 32:
    torch.set_default_dtype(torch.float32)
elif precision == 64:
    torch.set_default_dtype(torch.float64)
    data = rdata.double()

In [4]:
rdatamin = rdata.min()
data = torch.log10(rdata - rdatamin + 1)[::10]
mindata = data.min()
data = (data - data.mean())/data.std()

In [5]:
class lle_ds(Dataset):
    def __init__(self, data):
        super().__init__()
        self.nst, self.np = data.size()
        self.np = self.np//2
        self.slowtimes = torch.linspace(0, 1, self.nst).requires_grad_()
        self.phases = torch.linspace(0, 1, self.np).requires_grad_()
        self.lle_Ams = data.flatten()

    def __len__(self):
        return self.nst*self.np

    def __getitem__(self, index):
        idx_st = index // self.np
        idx_p = index % self.np
        Am = (self.lle_Ams[index], self.lle_Ams[index+self.np])
        return (self.slowtimes[idx_st], self.phases[idx_p]), Am

In [6]:
lr = 1e-2
bs = 300
nMLP = 32
lMLP = 1
loss_fn = F.mse_loss
D2, alpha, g, Ain, D_ini, D_spd = 3.383297709479104e+06, 1.214590581942685e+07, 0.053924555862390, 2.178441699484071e+12, 1.214590581942685e+08, 9.391607661535454e+14
zomfac_st, zomfac_p, zomfac_A = 5.173089105573258e-06, 2*torch.pi,  1
left_paras = torch.Tensor([1/zomfac_st, D2/zomfac_p**2/2])
right_paras = torch.Tensor([alpha, D_ini, zomfac_st*D_spd, g, Ain])

trnset = lle_ds(data)
right_paras.dtype

torch.float32

In [7]:
def gradient(u, x, order = 1):
    if order == 1:
        return torch.autograd.grad(u, x, grad_outputs=torch.ones_like(u),
                                   create_graph=True,
                                   only_inputs=True, allow_unused=True)[0]
    else:
        return gradient(gradient(u, x, order=1), x, order=order-1)

def cal_loss_equ(op, loss_fn, x, y):
    """
    Calculate the loss for the PINN based on the given PDEs.

    Parameters:
    - op: A tuple or list containing the predicted values of Ar and Ai from the model.
    - loss_fn: A PyTorch loss function, such as nn.MSELoss.
    - x: A tuple or list containing the spatial and temporal points for the PDEs.
    - y: A tuple or list containing the true values of Ar and Ai for the PDEs.
    - paras: A tuple or list containing the physical parameters D2, alpha, D, g, and Ain.
    - paras: A tuple or list containing the physical parameters D2, alpha, g, Ain, D_ini, D_spd.
    D2 = 3.383297709479104e+06, alpha = 1.214590581942685e+07, Delta = 1.214590581942685e+08, g=0.053924555862390, Ain=2.178441699484071e+12, 
    
    Returns:
    - The computed loss value for the PINN.
    """
    #print(y)
    opr = zomfac_A*op[:, 0] + mindata
    opi = zomfac_A*op[:, 1] + mindata
    Ar = zomfac_A*y[0] + mindata
    Ai = zomfac_A*y[1] + mindata
    #print(y[0], x[1])
    #print('Ar', Ar, Ar.dtype, '\nAi', Ai)
    # zoom factor
    left_PDEr = left_paras[0]*gradient(opr, x[0], order=1) + left_paras[1]*gradient(opi, x[1], order=2)
    left_PDEi = left_paras[0]*gradient(opi, x[0], order=1) - left_paras[1]*gradient(opr, x[1], order=2)
    #left_PDEi = zomfac_A/zomfac_st*gradient(op[:, 1], x[0], order=1) - (zomfac_A/zomfac_p*paras[0]/2) * gradient(op[:, 0], x[1], order=2)
    right_PDEr = - right_paras[0] * Ar + (right_paras[1] + right_paras[2]*x[0]) * Ai - right_paras[3] * (Ar**2 + Ai**2) * Ai + right_paras[4]
    right_PDEi = - right_paras[0] * Ai - (right_paras[1] + right_paras[2]*x[0]) * Ar + right_paras[3] * (Ar**2 + Ai**2) * Ar
    #right_PDEr = - zomfac_A*(paras[1] * y[0] + (paras[4] + paras[5]*zomfac_st*x[0]) * y[1] - paras[2] * (y[0]**2 + y[1]**2) * y[1]) + paras[3]
    #right_PDEi = - zomfac_A*(paras[1] * y[1] - (paras[4] + paras[5]*zomfac_st*x[0]) * y[0] + paras[2] * (y[0]**2 + y[1]**2) * y[0])
    return loss_fn(left_PDEi.view(-1), right_PDEi) + loss_fn(left_PDEr.view(-1), right_PDEr)

def cal_loss_data(op, loss_fn, y):   return loss_fn(op.view(-1), y)

In [8]:
class PINN(LightningModule):
    def __init__(self, lr=None, bs=None, 
                 nMLP=None, lMLP=None,
                 loss_fn=None,
                 trnset=None
                 ):

        super().__init__()
        self.learning_rate = lr
        self.batch_size = bs

        self.MLP = nn.Sequential(
            nn.Linear(2, nMLP),
            nn.ReLU(),
            nn.Linear(nMLP, nMLP),
            nn.ReLU(),
            nn.Linear(nMLP, 2),
        )
        self.loss_fn = loss_fn
        self.trnset = trnset

    def forward(self, st, p):
        bs  = st.size()
        x = torch.cat((st.view(bs[0], 1), p.view(bs[0], 1)), dim=1)
        MLPout = self.MLP(x)
        #MLPout (bs, sl, freqs)
        return MLPout
    
    def predict(self, x):
        MLPout = self.MLP(x)
        return MLPout

    def training_step(self, batch, batch_idx):
        (st, p), (Ar, Ai) = batch
        op = self.forward(st, p)
        # op (bs, 2)
        #PDE_loss = cal_loss_equ(op, self.loss_fn, (st, p), (Ar, Ai))
        
        if 20 > 10:
            loss = cal_loss_data(op[:, 0], self.loss_fn, Ar) + cal_loss_data(op[:, 1], self.loss_fn, Ai)
        else:
            loss = cal_loss_data(op[:, 0], self.loss_fn, Ar) + cal_loss_data(op[:, 1], self.loss_fn, Ai) + PDE_loss
        self.log('train_loss', loss)
        return loss
    
    def validation_step(self, batch, batch_idx):
        (st, p), (Ar, Ai) = batch
        op = self.forward(st, p)
        # op (bs, 2)
        loss = cal_loss_data(op[:, 0], self.loss_fn, Ar) + cal_loss_data(op[:, 1], self.loss_fn, Ai)
        self.log('val_loss', loss)
        return loss

    def train_dataloader(self):
        dataloader = DataLoader(self.trnset, batch_size=self.batch_size)
        return dataloader

    def val_dataloader(self):
        dataloader = DataLoader(self.trnset, batch_size=self.batch_size)
        return dataloader
    
    def configure_optimizers(self):
        from torch.optim import AdamW, Adam, SGD
        optim = Adam(self.parameters(), lr=self.learning_rate)
        return optim

In [9]:
Pmodel = PINN(lr, bs, 
                 nMLP, lMLP,
                 loss_fn, 
                 trnset
                 )

In [10]:
trainer = Trainer(max_epochs=2)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [11]:
trainer.fit(Pmodel)

You are using a CUDA device ('NVIDIA GeForce RTX 4060 Ti') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type       | Params | Mode 
--------------------------------------------
0 | MLP  | Sequential | 1.2 K  | train
--------------------------------------------
1.2 K     Trainable params
0         Non-trainable params
1.2 K     Total params
0.005     Total estimated model params size (MB)


Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

d:\anaconda\envs\dl\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:424: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


d:\anaconda\envs\dl\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


Epoch 2: 100%|██████████| 2134/2134 [00:24<00:00, 86.80it/s, v_num=1]

d:\anaconda\envs\dl\Lib\site-packages\lightning\pytorch\trainer\call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...
